# Module import

In [1]:
from ETGEMs_function_ETG import *

In [ ]:
import pandas as pd
import cobra

# Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original = './reaction_g0_ETC_butanol.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original = './metabolites_lnC_ETC_butanol.txt'
#Get Model from local file
model_file_original = './iML1515_butanol.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original = './ID_kcat_MW_file_ETC_butanol.csv'

In [4]:
Concretemodel_Need_Data_original=Get_Concretemodel_Need_Data(reaction_g0_file_original,metabolites_lnC_file_original,model_file_original,reaction_kcat_MW_file_original)

In [51]:
Concretemodel_Need_Data_original['ub_list']['EX_but_e'] = 0

Concretemodel_Need_Data_original['lb_list']['PPC'] = 0
Concretemodel_Need_Data_original['lb_list']['LDH_D'] = 0
Concretemodel_Need_Data_original['lb_list']['ACALD'] = 0
Concretemodel_Need_Data_original['lb_list']['PTAr'] = 0
Concretemodel_Need_Data_original['ub_list']['PPC'] = 0
Concretemodel_Need_Data_original['ub_list']['LDH_D'] = 0
Concretemodel_Need_Data_original['ub_list']['ACALD'] = 0
Concretemodel_Need_Data_original['ub_list']['PTAr'] = 0
Concretemodel_Need_Data_original['lb_list']['ACACT1r'] = 0
Concretemodel_Need_Data_original['lb_list']['HACD1'] = 0
Concretemodel_Need_Data_original['lb_list']['ECOAH1'] = 0
Concretemodel_Need_Data_original['ub_list']['ACACT1r'] = 0
Concretemodel_Need_Data_original['ub_list']['HACD1'] = 0
Concretemodel_Need_Data_original['ub_list']['ECOAH1'] = 0

#set the maximum of the intake rate for butyrate
Concretemodel_Need_Data_original['ub_list']['EX_but_e_reverse'] = 3/87 * 180

#set the gene expression level for the reaction of EX_but_e_reverse
#if biomass_id='BIOMASS_Ec_iML1515_core_75p37M', the code in the next line can't be run. If biomass_id='EX_btoh_e', the code in the next line must be run.
Concretemodel_Need_Data_original['lb_list']['BIOMASS_Ec_iML1515_core_75p37M'] =0.93*0.26 #0.19

# Data output files

In [52]:
#calculate the maximum of the target reaction fluxes
#if obj_name='BIOMASS_Ec_iML1515_core_75p37M', the max flux for BIOMASS_Ec_iML1515_core_75p37M will be calculated
#if obj_name='EX_btoh_e', the max flux for n-butanol production reaction will be calculated

#obj_name='BIOMASS_Ec_iML1515_core_75p37M'
obj_name='EX_btoh_e'
obj_target='maximize'
E_total=0.19
substrate_name='EX_glc__D_e_reverse'
substrate_value=8
K_value=1249

max_biomass_under_mdf_original=Max_Growth_Rate_Calculation_kinetics(Concretemodel_Need_Data_original,obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,'gurobi')
if obj_name == 'BIOMASS_Ec_iML1515_core_75p37M':
    print("Max biomass value_original : " +str(max_biomass_under_mdf_original))
elif obj_name == 'EX_btoh_e':
    mass = max_biomass_under_mdf_original/180*74
    print("Max biomass value_original : " +str(mass))

Max biomass value_original : 2.747165852356576


In [19]:
## Minimum flux sum calculation（pFBA）
biomass_id='EX_btoh_e'
E_total=0.18
substrate_name='EX_glc__D_e_reverse'
substrate_value=8

biomass_value_original=max_biomass_under_mdf_original*0.99

K_value=1249

B_value_original=B_value_original
[min_V_original,Concretemodel_original]=Min_Flux_Sum_Calculation(Concretemodel_Need_Data_original,biomass_value_original,biomass_id,substrate_name,substrate_value,K_value,E_total,B_value_original,'gurobi')
print("Min flux amount_original : " +str(min_V_original))

Min flux amount_original : 408.0930954579125


# Simulation results

In [20]:
model_original=Concretemodel_Need_Data_original['model']
reaction_kcat_MW_original=Concretemodel_Need_Data_original['reaction_kcat_MW']
reaction_g0_original=Concretemodel_Need_Data_original['reaction_g0']
coef_matrix_original=Concretemodel_Need_Data_original['coef_matrix']
metabolite_list_original=Concretemodel_Need_Data_original['metabolite_list']

use_result_original = Get_Results_Thermodynamics(model_original,Concretemodel_original,reaction_kcat_MW_original,reaction_g0_original,coef_matrix_original,metabolite_list_original)

use_result_original = use_result_original[use_result_original['flux'] > 1e-10] 
use_result_original = use_result_original.sort_values(by = 'flux',axis = 0,ascending = False)
use_result_original["reaction"] = use_result_original.apply(lambda row: model_original.reactions.get_by_id(row.name).reaction, axis = 1)
use_result_original["gpr"] = use_result_original.apply(lambda row: model_original.reactions.get_by_id(row.name).gene_reaction_rule, axis = 1)

In [21]:
use_result_tmp_original=use_result_original[use_result_original['f']>-1249]
use_result_select_original=use_result_tmp_original[abs(use_result_tmp_original['f']-B_value_original)<=1e-05]
use_result_select_original.head(10)

,flux,z,f,enz,met_concentration,reaction,gpr


In [22]:
#output the fluxes for acetate intake reaction and butyrate uptake reaction

print(use_result_original['flux']['EX_ac_e']*60/180)
print(use_result_original['flux']['EX_but_e_reverse']*87/180)

3.6293886443972783
5.98206517607308
